In [ ]:
import os

import pandas
import numpy
from keras_preprocessing import image
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
base_dir = r"D:\user's\work related\to deepsight ai"
learning_rate = 0.001
epochs = 20
epoch_steps = 50

In [ ]:
# [project folder] "base directory"/..
#     [subfolder] Training/..
#         [subfolder] class_0/..
#         [subfolder] class_1/..
#         [subfolder] ..
#     [subfolder] Testing/..
#         [subfolder] Testing/..
#     [subfolder] Validation/..
#         [subfolder] class_0/..
#         [subfolder] class_1/..
#         [subfolder] ..

In [ ]:
train_dir = os.path.join(base_dir, 'Training')
validation_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Testing')

In [ ]:
train_datagenerator = ImageDataGenerator(rescale=1. / 255., rotation_range=40, width_shift_range=0.2,
                                         height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagenerator = ImageDataGenerator(rescale=1. / 255.)
train_generator = train_datagenerator.flow_from_directory(train_dir, batch_size=20, class_mode='categorical', color_mode='rgb', shuffle=True,
                                                          target_size=(256, 256))
validation_generator = validation_datagenerator.flow_from_directory(validation_dir, batch_size=20, class_mode='categorical', color_mode='rgb', shuffle=True,
                                                                    target_size=(256, 256))

test_datagenerator = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagenerator.flow_from_directory(test_dir, target_size=(256, 256), batch_size=1,class_mode=None, shuffle=False)

In [ ]:
base_vgg16model = VGG16(input_shape=(256, 256, 3), include_top=False, weights='imagenet', pooling='max')

for layer in base_vgg16model.layers:
    layer.trainable = False

output_layer = layers.Flatten()(base_vgg16model.output)
output_layer = layers.Dense(1024, activation='relu')(output_layer)
output_layer = layers.Dropout(0.2)(output_layer)
output_layer = layers.Dense(6, activation='softmax')(output_layer)

vgg16_model = tf.keras.models.Model(base_vgg16model.input, output_layer)
vgg16_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate), metrics=['acc'],
                    loss=tf.keras.losses.categorical_crossentropy)

vgghist = vgg16_model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=epoch_steps, epochs=epochs)

In [ ]:
base_inceptionmodel = InceptionV3(input_shape = (256, 256, 3), include_top=False, weights='imagenet', pooling='max')

for layer in base_inceptionmodel.layers:
    layer.trainable = False

output_layer = layers.Flatten()(base_inceptionmodel.output)
output_layer = layers.Dense(1024, activation='relu')(output_layer)
output_layer = layers.Dropout(0.2)(output_layer)
output_layer = layers.Dense(6, activation='softmax')(output_layer)

inception_model = tf.keras.models.Model(base_inceptionmodel.input, output_layer)
inception_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate), metrics=['acc'],
                    loss=tf.keras.losses.categorical_crossentropy)

inceptionhist = inception_model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=epoch_steps, epochs=epochs)

In [ ]:
base_resnetmodel = ResNet50(input_shape=(256, 256,3), include_top=False, weights="imagenet", pooling='max')

for layer in base_resnetmodel.layers:
    layer.trainable = False
    
resnet_model = Sequential()
resnet_model.add(base_resnetmodel)
resnet_model.add(layers.Dense(6, activation='softmax'))
resnet_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate), metrics=['acc'],
                    loss=tf.keras.losses.categorical_crossentropy)

resnethist = resnet_model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=epoch_steps, epochs=epochs)

In [ ]:
base_efficientmodel = EfficientNetB0(input_shape=(256, 256, 3), include_top=False, weights='imagenet', pooling='max')

for layer in base_efficientmodel.layers:
    layer.trainable = False
    
output_layer = base_efficientmodel.output
output_layer = layers.Flatten()(output_layer)
output_layer = layers.Dense(1024, activation="relu")(output_layer)
output_layer = layers.Dropout(0.2)(output_layer)
output_layer = layers.Dense(6, activation="softmax")(output_layer)

efficient_model = tf.keras.models.Model(base_efficientmodel.input, output_layer)
efficient_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate), metrics=['acc'],
                    loss=tf.keras.losses.categorical_crossentropy)

efficienthist = efficient_model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=epoch_steps, epochs=epochs)

In [ ]:
classes = list(train_generator.class_indices.keys())

In [ ]:
testset = pandas.read_csv(base_dir + '/test_set.csv')

for row in range(len(testset)):
    filename = testset.iloc[row, 0]
    test_image = image.load_img(test_dir + "/" + filename, target_size=(256, 256))
    test_image = image.img_to_array(test_image)
    test_image = numpy.expand_dims(test_image, axis=0)
    # chosen InceptionV3 model as it has high accuracy
    pred = inception_model.predict(test_image)
    result = pred.round()[0]
    class_index = numpy.argmax(result, axis=0)
    testset.iloc[row, 1] = classes[class_index]

In [ ]:
testset.to_csv('test_set.csv', index=False)